In [8]:
!pip install pandas ultralytics tensorflow

In [9]:
import os
HOME = os.getcwd()
print(HOME)

/Users/phingphing/Downloads/Driver-drowsiness-detection-main-2/tranining_model_manual


In [10]:
pip install ultralytics


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.203 🚀 Python-3.10.11 torch-2.8.0 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 123.9/228.3 GB disk)


### Traning model

In [ ]:
from ultralytics import YOLO

# โหลดโมเดล YOLOv9 (เลือกขนาด: yolov9n, yolov9s, yolov9m, yolov9l, yolov9e)
# โหลดโมเดล YOLOv8 (เลือกขนาด: yolov8n)
model = YOLO("yolo11m.pt") # c = compact, e = extra large

# เทรน
model.train(
    data="pool_data_set/data_set_1/data.yaml",   # path ไปที่ไฟล์ yaml
    epochs=50,                   # จำนวนรอบการเทรน
    imgsz=640,                   # ขนาดภาพ
    batch=32,                    # batch size
    workers=3,                   # worker สำหรับโหลดข้อมูล
    device=0,                     # ใช้ GPU (0) ถ้าไม่มีให้ใส่ 'cpu'
    project="model",
    name="model"
)


New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.189  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model9_yolov11n_v1i2, nbs=64, nms=False, opset=None, opt

RuntimeError: Dataset '../data/data.yaml' error  '../data/data.yaml' does not exist

# Test model open camera

In [ ]:
import cv2
import time
from ultralytics import YOLO

# โหลดโมเดล YOLO ที่เทรนสำหรับ face + eye
# (ต้องมี class "face", "open_eye", "closed_eye")
model = YOLO("/Users/phingphing/Downloads/Driver-drowsiness-detection-main-2/data/yolo.pt")

# เปิดกล้อง
cap = cv2.VideoCapture(0)

drowsy_start_time = None
alert_threshold = 5  # วินาที

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects (face + eyes)
    results = model.predict(frame, imgsz=640, conf=0.7)  # เพิ่ม conf=0.7
    detections = results[0].boxes.data.cpu().numpy()

    is_drowsy = False
    face_detected = False

    # Debug: พิมพ์ค่าที่ตรวจจับได้
    print(f"Detections: {detections}")

    for detection in detections:
        x1, y1, x2, y2, conf, class_id = detection
        class_id = int(class_id)

        # สมมติ class 0 = face, 1 = open_eye, 2 = closed_eye
        if class_id == 0:  
            face_detected = True
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        elif class_id == 1:  # closed eye
            cv2.putText(frame, "Eye: Closed", (int(x1), int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            is_drowsy = False  # รีเซ็ตสถานะหากตรวจพบ open_eye

        elif class_id == 2:  #   open eye
            cv2.putText(frame, "Eye: Open", (int(x1), int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            is_drowsy = True

    status = "DROWSY" if is_drowsy else "ALERT"

    # ถ้าตาปิดนานเกิน threshold → แจ้งเตือน
    if is_drowsy and face_detected:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time > alert_threshold:
            cv2.putText(frame, "ALERT: DROWSY!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    else:
        drowsy_start_time = None

    # แสดงสถานะ
    cv2.putText(frame, f"Status: {status}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if status=="ALERT" else (0,0,255), 2)

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 335.9ms
Speed: 6.9ms preprocess, 335.9ms inference, 10.6ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 389.5ms
Speed: 2.3ms preprocess, 389.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 279.9ms
Speed: 2.0ms preprocess, 279.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 191.0ms
Speed: 2.3ms preprocess, 191.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 202.2ms
Speed: 2.8ms preprocess, 202.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 187.9ms
Speed: 1.6ms preprocess, 187.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Detections: []

0: 384x640 (no detections), 203.3ms
Speed: 1.7ms preprocess, 203.3ms inference, 0.3ms postprocess 

In [ ]:
import cv2
import time
from ultralytics import YOLO

# โหลดโมเดล YOLO ที่เทรนสำหรับ face + eye
# (ต้องมี class "face", "open_eye", "closed_eye")
model = YOLO("/Users/phingphing/Downloads/Driver-drowsiness-detection-main-2/data/yolo.pt")

# เปิดกล้อง
cap = cv2.VideoCapture(0)

drowsy_start_time = None
alert_threshold = 5  # วินาที

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects (face + eyes)
    results = model.predict(frame, imgsz=640, conf=0.7)  # เพิ่ม conf=0.7
    detections = results[0].boxes.data.cpu().numpy()

    is_drowsy = False
    face_detected = False

    # Debug: พิมพ์ค่าที่ตรวจจับได้
    print(f"Detections: {detections}")

    for detection in detections:
        x1, y1, x2, y2, conf, class_id = detection
        class_id = int(class_id)

        # สมมติ class 0 = face, 1 = open_eye, 2 = closed_eye
        if class_id == 0:  
            face_detected = True
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        elif class_id == 1:  # open eye
            cv2.putText(frame, "Eye: Open", (int(x1), int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            is_drowsy = False  # รีเซ็ตสถานะหากตรวจพบ open_eye

        elif class_id == 2:  # closed eye
            cv2.putText(frame, "Eye: Closed", (int(x1), int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            is_drowsy = True

    status = "ALERT" if is_drowsy else "DROWSY"

    # ถ้าตาปิดนานเกิน threshold → แจ้งเตือน
    if is_drowsy and face_detected:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time > alert_threshold:
            cv2.putText(frame, "DROWSY: ALERT!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    else:
        drowsy_start_time = None

    # แสดงสถานะ
    cv2.putText(frame, f"Status: {status}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if status=="DROWSY" else (0,0,255), 2)

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Drinking, 250.2ms
Speed: 5.9ms preprocess, 250.2ms inference, 10.3ms postprocess per image at shape (1, 3, 384, 640)
Detections: [[     157.35    0.056396        1920        1080      0.9653           2]]

0: 384x640 1 Drinking, 189.2ms
Speed: 4.2ms preprocess, 189.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Detections: [[     51.893     0.80099      1477.6      1079.7     0.94276           2]]

0: 384x640 1 Drinking, 165.5ms
Speed: 1.3ms preprocess, 165.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Detections: [[     25.052      1.0975      1476.1      1079.7     0.94454           2]]

0: 384x640 1 Drinking, 279.1ms
Speed: 1.5ms preprocess, 279.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Detections: [[          0    0.010895      1478.5        1080     0.94906           2]]

0: 384x640 1 Drinking, 171.7ms
Speed: 1.5ms preprocess, 171.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 64

# Convert model for TFlite for use in Android

In [9]:
from ultralytics import YOLO

# Load your custom trained model
model = YOLO("model/model8_yolov11n_v1i_1/weights/best.pt")

# Export to TFLite format
model.export(format="tflite", imgsz=640)



Ultralytics 8.3.202 🚀 Python-3.10.18 torch-2.8.0 CPU (Apple M4)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'model/model8_yolov11n_v1i_1/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success ✅ 1.1s, saved as 'model/model8_yolov11n_v1i_1/weights/best.onnx' (10.2 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.2...
Saved artifact at 'model/model8_yolov11n_v1i_1/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 5, 8400), dtype=tf.float32, name=None)
Captures:
  4420433728: TensorSpec(shape=(4, 2), dtype=tf.int32,

I0000 00:00:1758211111.003499   57333 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1758211111.003562   57333 single_machine.cc:374] Starting new session
W0000 00:00:1758211111.709325   57333 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1758211111.709336   57333 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1758211112.084911   57333 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1758211112.084971   57333 single_machine.cc:374] Starting new session


TensorFlow SavedModel: export success ✅ 7.5s, saved as 'model/model8_yolov11n_v1i_1/weights/best_saved_model' (25.6 MB)

TensorFlow Lite: starting export with tensorflow 2.19.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite' (10.1 MB)

Export complete (7.6s)
Results saved to /Users/tutamini/Documents/Github/TrainPotHole/model/model8_yolov11n_v1i_1/weights
Predict:         yolo predict task=detect model=model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite imgsz=640 data=pothole.v1i.yolov8/data.yaml  
Visualize:       https://netron.app


W0000 00:00:1758211112.843207   57333 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1758211112.843217   57333 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


'model/model8_yolov11n_v1i_1/weights/best_saved_model/best_float32.tflite'

In [ ]:
import tensorflow as tf

# โหลดโมเดล SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model("model/model1/weights/best.pt")

# (optional) ลดขนาดโมเดล
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# แปลงเป็น TFLite
tflite_model = converter.convert()

# บันทึกไฟล์
with open("model1.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ แปลงเสร็จแล้ว: model1.tflite")
